Importing libraries

In [1]:
import os

import cv2
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
from keras import Model
from keras.applications import mobilenet_v2
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Dense
from keras.preprocessing import image
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


Loading files with paths of pictures and their categories

In [2]:
#current_file = os.path.abspath(os.path.dirname(__file__))

TRAIN_LABELS_FILE = (r'./drugs/train/labels.txt')
VAL_LABELS_FILE = (r'./drugs/val/labels.txt')
TEST_LABELS_FILE = (r'./drugs/test/labels.txt')

Creating augmenting function for better model performing

In [3]:
def augment(img):
    seq = iaa.Sequential([
        iaa.Crop(px=(0, 16)),  # croping images from each side by 0 to 16px (randomly chosen)
        iaa.Fliplr(0.5),  # flipping horizontally 50% of the images
        iaa.GaussianBlur(sigma=(0, 3.0))  # bluring images with a sigma of 0 to 3.0
    ])
    img = seq(images=np.array([img]))[0]
    return mobilenet_v2.preprocess_input(img)

Creating generator for images loading

In [4]:
def get_generator(filename, number=None, shuffle=True):
    df = pd.read_csv(filename, delimiter=' ', names=["path", "value"],dtype="str")
    if number:
        df = df[:number]
    
    gen = image.ImageDataGenerator(preprocessing_function=mobilenet_v2.preprocess_input)
 
    directory = os.path.dirname(filename)
    
    return gen.flow_from_dataframe(df, directory, "path", "value", target_size=(224, 224), batch_size=16, shuffle=shuffle) 

Imagine showing

In [5]:
def show_dataset(flow):
    for img, l in flow:
        print(l[0])
        
        img = img[0].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        cv2.imshow("image", img)
        
        key = cv2.waitKey(0)
        if key == 27: #if ESC is pressed, exit
            cv2.destroyAllWindows()
            break

In [6]:
def show_test():
    train_gen = get_generator(TRAIN_LABELS_FILE,10) #showing images for only 5 classes, it's only for checking purposes
    show_dataset(train_gen)

In [7]:
#show_test()

Creating a model based on MobileNetV2

In [8]:
def create_model(gen):
    
    model = mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), include_top=False, pooling="avg", weights="imagenet")
    dense = Dense(len(gen.class_indices), activation="softmax")(model.output)
    model = Model(inputs=[model.input], outputs=[dense])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["categorical_accuracy"])

    return model

Creating function for getting callback to visuale learing process, performance in Tensorboard, implemanting model checkpoints and early stopping

In [9]:
def get_callbacks():
    
    callbacks = []

    mc = ModelCheckpoint("weights.hdf5", monitor="val_loss", save_best_only=True, verbose=1)
    es = EarlyStopping(monitor="val_loss", patience=10, verbose=1)
    tb = TensorBoard()

    callbacks.append(mc)
    callbacks.append(es)
    callbacks.append(tb)

    return callbacks

Function for testing model

In [10]:
def test_model(model):

    test_gen = get_generator(TRAIN_LABELS_FILE)

    results = model.evaluate_generator(test_gen, steps=test_gen.n // test_gen.batch_size)
   
    for name, result in zip(model.metrics_names, results):
        print("{}: {}".format(name, result))

Function for getting labels to numpy array (n-rows (pictures count) x 20 columns(classes))

In [11]:
def get_gt_from_generator(gen):
    gt = []
    
    for _ in range(len(gen)):
        _, labels = next(gen)
        gt.extend(labels)

    gen.reset()

    return np.array(gt).astype(np.uint8)

Creating a report on testing data

In [15]:
def show_report(model):
    
    gen = get_generator(TEST_LABELS_FILE, shuffle=False)

    gt = get_gt_from_generator(gen)
    pred = model.predict_generator(gen)
    pred = np.round(pred).astype(np.uint8)
    report = classification_report(gt, pred)

    print(report)

Main fuction for starting whole project

In [16]:
def main():
    # traing data
    train_gen = get_generator(TRAIN_LABELS_FILE)

    # validation data
    val_gen = get_generator(VAL_LABELS_FILE)

    # callbacks
    callbacks = get_callbacks()

    # model creation
    model = create_model(train_gen)
    
    # model summary
    model.summary()

    # model fitting 
    model.fit_generator(train_gen, steps_per_epoch=train_gen.n // train_gen.batch_size, epochs=100, callbacks=callbacks, validation_data=val_gen, validation_steps=val_gen.n // val_gen.batch_size)

    # model testing
    test_model(model)

    # report
    show_report(model)

In [17]:
main()

roject[0][0]           
__________________________________________________________________________________________________
block_11_add (Add)              (None, 14, 14, 96)   0           block_10_project_BN[0][0]        
                                                                 block_11_project_BN[0][0]        
__________________________________________________________________________________________________
block_12_expand (Conv2D)        (None, 14, 14, 576)  55296       block_11_add[0][0]               
__________________________________________________________________________________________________
block_12_expand_BN (BatchNormal (None, 14, 14, 576)  2304        block_12_expand[0][0]            
__________________________________________________________________________________________________
block_12_expand_relu (ReLU)     (None, 14, 14, 576)  0           block_12_expand_BN[0][0]         
_____________________________________________________________________________________